<h1>Dependencies</h1>

In [1]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredWordDocumentLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
import pypdf

<h1>Embed Class Slides and Textbooks</h1>

In [ ]:
# Load documents
loader = DirectoryLoader('../docs', glob="**/*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = Chroma.from_documents(persist_directory='../chroma', documents=splits, embedding=embeddings)

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong poi

<h1>Initialize Llama LLM locally</h1>

In [2]:
embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = Chroma(persist_directory='../chroma', embedding_function=embeddings)
# Initialize LLama 3.2
llm = Ollama(model="llama3.2", base_url="http://localhost:11434")



/tmp/ipykernel_482289/2116260297.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory='../chroma', embedding_function=embeddings)
/tmp/ipykernel_482289/2116260297.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2", base_url="http://localhost:11434")


In [4]:
prompt = 'Generate a multiple choice network security quiz.'

similar_docs = vectorstore.similarity_search(prompt)
for document in similar_docs:
    print(f"Source: {document.metadata['source']} Page Number: {document.metadata['page']}")
context = "\n".join([doc.page_content for doc in similar_docs])
response = llm(f"Context: {context}\nQuestion: {prompt}\nAnswer:")

Source: ../docs/ComputerNetworkingTopDownApproach.pdf Page Number: 488
Source: ../docs/Lecture_2_slides.pdf Page Number: 0
Source: ../docs/book-perlner-network-security-erb.pdf Page Number: 388
Source: ../docs/ComputerNetworkingTopDownApproach.pdf Page Number: 147


In [5]:
response

'I can help you with each of these questions.\n\n1. Can an encryption algorithm map a block with given value x to a block with given value y?\n\nThe answer to this question depends on the specific encryption algorithm being used. In general, encryption algorithms are designed to be one-way functions, meaning that it is difficult to map a block back to its original input (x). However, some encryption algorithms may allow for reversible encryption, where an encrypted block can be decrypted back into the original input.\n\n2. Which of the following are possible in an encryption algorithm with decryption ability?\n\na) Two tuples mapping to the same ciphertext block:\n\nYes, this is possible. If two different keys produce the same ciphertext output, then it is likely that the encryption algorithm is vulnerable to a type of attack known as "collision".\n\nb) Two tuples with the same key producing different ciphertext blocks:\n\nThis is also possible. If an attacker can find a way to map dif

<h1>Generate prompt with GUI using gradio</h1>

In [ ]:
import gradio as gr
import random

def answer(message: str, history: list[str]):
    prompt = "\n".join([f"User: {item[0]}\nBot: {item[1]}" for item in history]) + f"\nUser: {message}\nBot:"
    similar_docs = vectorstore.similarity_search(prompt)

    source = "\n".join([f"Source: {document.metadata['source']} Page Number: {document.metadata['page']}" for document in similar_docs])

    context = "\n".join([doc.page_content for doc in similar_docs])
    response = llm(f"Context: {context}\nQuestion: {prompt}\nAnswer:")
    history.append([message, response])

    return (response + "\n\n" + source)


chatbot = gr.ChatInterface(
    fn=answer,
    title="Group 5 Quizbot",
    description="Enter the following prompt to be quizzed: Generate a multiple choice, true/false, or short answer network security question and provide feedback to my answer."
)

chatbot.launch(server_port=7860)


/home/nathan/GIT_branches/CS5342Project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/tmp/ipykernel_482289/3907343863.py:11: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(f"Context: {context}\nQuestion: {prompt}\nAnswer:")
